In [1]:
# !pip install -q pandas openpyxl

In [2]:
import pandas as pd
import random

# 1. Excel 파일에서 특정 시트 읽기
file_path = "P3P4_단답형 문제_라벨링_0.1.xlsx"
sheet_name = "P3P4_단답형 문제"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# 2. 특정 컬럼 선택
column_name = "question"  # 추출하고 싶은 컬럼명
values = df[column_name].dropna().tolist()  # 결측값 제거 후 리스트로 변환

# 3. 랜덤으로 N개 샘플링
random.seed(2025)

num_samples = 10  # 추출할 개수
sampled_values = random.sample(values, k=num_samples)

# 결과 확인
for i, value in enumerate(sampled_values):
    print(f"Q{i + 1}: {value}")

Q1: 어떤 가분수의 분자를 $8$로 나누었더니 몫이 $2$이고 나머지가 $2$보다 큽니다. 또 이 분수의 분모를 $3$으로 나누었더니 몫이 $1$이고, 나머지가 있습니다. 조건을 만족하는 가분수 중에서 $4$보다 작은 가분수를 구하시오.
Q2: $\square$ 안에 알맞은 수를 써넣으시오. $ 80 \div 6= \square···\square$ (검산) $ 6 \times \square+\square=\square $
Q3: 장미를 $8$개의 꽃병에 똑같게 나누어 꽂았습니다. 장미가 모두 $48$송이라면 꽃병 하나에 몇 송이씩 꽂았습니까?
Q4: 색 테이프를 민호는 $0.385 m$, 나영이는 $26 cm$, 석현이는 $0.29 m$ 가지고 있습니다. 누가 가진 색 테이프의 길이가 가장 짧습니까?
Q5: $9600$억에서 $1000$억씩 $5$번 뛰어서 센 수를 쓰시오.
Q6: 승현이는 우표를 $226$장, 태희는 $178$장 모았습니다. 두 사람이 모은 우표는 모두 몇 장입니까?
Q7: $ \frac{\triangle}{13} $ 모양의 분수 중에서 $ \frac{1}{13} $이 $5$개인 분수보다 크고 $ \frac{9}{13} $보다 작은 분수를 모두 쓰시오.
Q8: 두 변의 길이가 다음과 같은 이등변삼각형을 그리려고 합니다. 나머지 한 변이 될 수 있는 길이를 모두 쓰시오. $7cm,12cm$
Q9: 두 분수의 크기를 비교하여 $\bigcirc$ 안에 >, =, <를 알맞게 써넣으시오. (1) $ \frac{2}{7} \bigcirc \frac{5}{7} $ (2) $ \frac{1}{4} \bigcirc \frac{1}{5} $
Q10: 어떤 수에 $17$을 곱해야 할 것을 잘못하여 $17$을 뺐더니 $35$가 되었습니다. 바르게 계산하면 얼마입니까?


In [3]:
# !pip install -q torch accelerate

In [4]:
# !pip install -q git+https://github.com/lgai-exaone/transformers@add-exaone4

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "LGAI-EXAONE/EXAONE-4.0-1.2B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to("cuda")

/home/jungi.byun/AMA/solution/math_agent/math_agent_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
for question in sampled_values:
    messages = [
        {
            "role": "system",
            "content": """주어진 {question}을 한국어로 풀이 및 정답을 간결하게 설명해주세요.
            \square, \triangle, \bigcirc 은 실제 해당 도형이 아닌, 문제의 정답을 위한 숫자 또는 기호를 채워야하는 부분으로 인식하세요.
            ## 문제 이해 ##
                - 주어진 {question}을 풀기위한 전략을 간단하게 세워보세요.
            ## 풀이 ##
                - 수립한 전략을 토대로 차근차근 단계별로 나누어 풀어주세요.
            ## 정답 ##
                - 주어진 {question}에 대한 정답을 명료하게 제공해주세요.
            """
        },
        {
            "role": "user",
            "content": question
        }
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        enable_thinking=True,
    )
    
    output = model.generate(
        input_ids.to(model.device),
        max_new_tokens=16384,
        do_sample=True,
        temperature=0.3,
        top_p=0.95
    )
    
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    THINK_START_TOKEN = "<think>"
    THINK_END_TOKEN = "</think>"

    if decoded.find(THINK_END_TOKEN) != -1:
        sol_start = decoded.find(THINK_END_TOKEN) + len(THINK_END_TOKEN)
    elif decoded.find(THINK_START_TOKEN) != -1:
        sol_start = decoded.find(THINK_START_TOKEN) + len(THINK_START_TOKEN)
    else:
        sol_start = 0
    print(f"Q: {question}")
    print(f"A: {decoded[sol_start:]}")
    # print(f"A: {decoded}")
    print("=" * 50)

Q: 어떤 가분수의 분자를 $8$로 나누었더니 몫이 $2$이고 나머지가 $2$보다 큽니다. 또 이 분수의 분모를 $3$으로 나누었더니 몫이 $1$이고, 나머지가 있습니다. 조건을 만족하는 가분수 중에서 $4$보다 작은 가분수를 구하시오.
A: 

주어진 조건을 만족하는 가분수를 찾기 위해 다음과 같이 단계적으로 접근한다.

- 분수를 \( \frac{a}{b} \)로 표현하고, 가분수 조건에 따라 \( a > b \)이다.
- 분자 \( a \)를 8로 나눈 몫이 2이고 나머지가 2보다 크므로, \( a = 16 + r_a \)이며 \( r_a \geq 3 \) (나머지는 0 이상 7 이하).
- 분모 \( b \)를 3으로 나눈 몫이 1이고 나머지가 있으므로, \( b = 3 + r_b \)이며 \( r_b = 1 \) 또는 \( 2 \) (나머지는 0이 아님).
- 따라서 \( b = 4 \) 또는 \( 5 \)이다.
- 분수 값이 4보다 작으므로 \( \frac{a}{b} < 4 \).

각 경우를 검토한다:
- \( b = 4 \)일 때:
  - \( \frac{a}{4} < 4 \) ⇒ \( a < 16 \).
  - \( a = 16 + r_a \geq 19 \)이므로 \( a < 16 \)과 모순. 불가능.
- \( b = 5 \)일 때:
  - \( \frac{a}{5} < 4 \) ⇒ \( a < 20 \).
  - \( a = 16 + r_a \geq 19 \)이므로 \( 19 \leq a < 20 \).
  - 정수 조건에 따라 \( a = 19 \), \( r_a = 3 \) (2보다 큼).
  - \( b = 5 \), \( r_b = 2 \) (0이 아님).
  - 분수 \( \frac{19}{5} = 3.8 < 4 \)이고 \( 19 > 5 \)로 가분수 조건 만족.

다른 가능성은 없으며, 유일한 해는 \( \frac{19}{5} \)이다.

\boxed{\dfrac{19}{5}}
Q: $\square$ 안에 알맞은 수를 써넣으시오